<a href="https://colab.research.google.com/github/HTTPArchive/almanac.httparchive.org/blob/main/src/tools/scripts/chart-adjustments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Authenticate the user
import sys

if 'google.colab' in sys.modules:
    from google.colab import auth

    auth.authenticate_user()
    credentials = auth.get_user_credentials()
else:
    import google.auth

    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets'
    ]
    credentials, project = google.auth.default(scopes=SCOPES)

In [ ]:
from googleapiclient.discovery import build

sheets_service = build('sheets', 'v4', cache_discovery=False, credentials=credentials)

def update_chart_size(spreadsheet_id, is_dry_run=False, target_width=600, target_height=371):
    response = sheets_service.spreadsheets().get(spreadsheetId=spreadsheet_id, includeGridData=False).execute()
    sheets = response.get('sheets', [])

    for sheet in sheets:
        charts = sheet.get('charts', [])

        for chart in charts:
            chart_id = chart['chartId']
            update_request = {
                'updateEmbeddedObjectPosition': {
                    'objectId': chart_id,
                    'newPosition': {
                        'overlayPosition': {
                            'widthPixels': target_width
                        }
                    },
                    'fields': 'widthPixels'
                }
            }

            if target_height is not None:
                update_request['updateEmbeddedObjectPosition']['newPosition']['overlayPosition']['heightPixels'] = target_height
                update_request['updateEmbeddedObjectPosition']['fields'] += ',heightPixels'

            if (chart['position']['overlayPosition']['widthPixels'] != target_width or
                 (target_height is not None and chart['position']['overlayPosition']['heightPixels'] != target_height)):
                if not is_dry_run:
                    sheets_service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body={'requests': [update_request]}).execute()

                print(f"""sheet: {sheet['properties']['title']},
chart: {chart['spec']['title']},
dimensions: {chart['position']['overlayPosition']['widthPixels']} x {chart['position']['overlayPosition'].get('heightPixels', 'N/A')}
                """)

In [4]:
# Replace this with the ID of your Google Sheets file
SPREADSHEET_ID = '1Svyw40Th7VbigX6lpR1lb1WXwTUVKZWrK7O2YELrml4'

# Call the function to update the chart width
update_chart_size(SPREADSHEET_ID, target_height=None, is_dry_run=False)

sheet: Cookies Third Party Top,
chart: Most common third-party cookies,
dimensions: 700 x 371
                
sheet: Cookie Domains Third Party Top,
chart: Most common third-party cookie domains,
dimensions: 538 x 371
                
sheet: Client Hints Top,
chart: Top client hints,
dimensions: 581 x 371
                
sheet: Referrer Policy Top,
chart: Most common referrer policies,
dimensions: 581 x 371
                
sheet: Referrer Policy Usage,
chart: Referrer policy usage,
dimensions: 581 x 371
                
sheet: Iab Tcf V2 Countries Top,
chart: IAB TCF v2, top publisher countries,
dimensions: 581 x 376
                
sheet: Dnt Usage,
chart: DoNotTrack usage,
dimensions: 581 x 376
                
sheet: Iab Usp Strings Top,
chart: Most common IAB USP string values,
dimensions: 581 x 459
                
sheet: Related Origin Trials Top,
chart: Most common privacy related Origin Trials,
dimensions: 581 x 459
                
